In [15]:
import numpy as np
import pandas as pd
from carbon_models import *
pd.options.plotting.backend = "plotly"

In [16]:
# Import data
df_carbon = pd.read_csv("carbon_data.csv",sep=",")
df_carbon.columns

Index(['Date', 'Region', 'saison', 'week_day', 'is_holiday', 'is_bank_holiday',
       'Temp', 'Temp1', 'Temp2', 'Temp3', 'Temp4', 'Temp5', 'Temp6', 'Temp7',
       'ce', 'ce1', 'ce2', 'ce3', 'ce4', 'ce5', 'ce6', 'ce7', 'te', 're',
       'Emissions'],
      dtype='object')

In [17]:
# Define parameters
train_start = '2019-01-01'
train_end = '2019-10-01'
test_end = '2019-12-31'

best_vars = [
    "Region",
    "Temp",
    "saison",
    "week_day",
    "is_holiday",
    "is_bank_holiday",
    "ce",
    "te",
    "re",
]

In [18]:
# Train the model and predict
# output_df = LSTM_predict(df_carbon,train_start,train_end,test_end)
# output_df = Catboost_predict(df_carbon,train_start,train_end,test_end,best_vars)
output_df = Xgboost_predict(df_carbon,train_start,train_end,test_end,best_vars)

In [19]:
output_df

,Date,Region,y,y_pred
0,2019-01-01,Auvergne-Rhône-Alpes,0.006316,NaN
1,2019-01-01,Bourgogne-Franche-Comté,0.002170,NaN
2,2019-01-01,Bretagne,0.002890,NaN
3,2019-01-01,Centre-Val de Loire,0.001938,NaN
4,2019-01-01,Grand Est,0.004186,NaN
...,...,...,...,...
4375,2019-12-31,Nouvelle-Aquitaine,0.006660,0.009524
4376,2019-12-31,Occitanie,0.006430,0.007805
4377,2019-12-31,Pays de la Loire,0.004118,0.005174
4378,2019-12-31,Provence-Alpes-Côte d'Azur,0.006005,0.007612


In [25]:
# Plot for the whole country
output_df.groupby(['Date']).apply(pd.DataFrame.sum,skipna=False).drop(columns=['Date','Region']).plot(title='France').update_layout(
    xaxis_title="Date", yaxis_title="Emissions (Mt/CO2)"
)

In [24]:
# Plot for the region `reg`
REGIONS = np.unique(df_carbon['Region'])
reg = REGIONS[8]

output_df[output_df['Region'] == reg].drop(columns='Region').set_index('Date').plot(title=reg).update_layout(
    xaxis_title="Date", yaxis_title="Emissions (Mt/CO2)"
)

In [22]:
is_test = (output_df['Date'] > train_end) * (output_df['Date'] <= test_end)

y_test = output_df['y'][is_test]
y_pred = output_df['y_pred'][is_test]

In [26]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

# Evaluate using Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

# Evaluate using Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Evaluate using Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")

# Evaluate using Coefficient of Determination (R^2)
r2 = r2_score(y_test, y_pred)
print(f"Coefficient of Determination (R^2): {r2}")

# Evaluate using Mean Absolute Percentage Error (MAPE)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"Mean Absolute Percentage Error (MAPE): {mape}")

Mean Squared Error (MSE): 5.1271428431731785e-06
Root Mean Squared Error (RMSE): 0.0022643195099572803
Mean Absolute Error (MAE): 0.0016920574010946604
Coefficient of Determination (R^2): 0.8008904365992807
Mean Absolute Percentage Error (MAPE): 0.2683815097411094
